In [2]:
import os
import pandas as pd
from collections import Counter
from kiwipiepy import Kiwi
import json

# 데이터셋 경로 설정
data_folder = "../Completed_csv"

# 형태소 분석기 초기화
kiwi = Kiwi()

# 불용어 리스트
custom_stopwords = [
    "지난해", "이날", "오늘", "내일", "올해", "시간", "관련", 
    "있다", "없다", "위해", "대한", "그리고", "등", "수", "것"
]

# 명사 추출
def extract_nouns(text):
    results = []
    analysis = kiwi.analyze(text)
    for token, pos, _, _ in analysis[0][0]:
        if len(token) > 1 and (pos.startswith('N') or pos.startswith('SL')) and token not in custom_stopwords:
            results.append(token)
    return results

# CSV 파일 로드 및 텍스트 결합
def load_and_combine_text(data_folder):
    combined_texts = []
    all_data = pd.DataFrame()

    for filename in os.listdir(data_folder):
        if filename.endswith(".csv"):
            filepath = os.path.join(data_folder, filename)
            df = pd.read_csv(filepath, encoding="utf-8")
            if "title" in df.columns and "short_content" in df.columns:
                df["combined"] = df["title"].fillna("") + " " + df["short_content"].fillna("")
                combined_texts.extend(df["combined"].tolist())
                all_data = pd.concat([all_data, df], ignore_index=True)

    return combined_texts, all_data

# 키워드 추출
def extract_top_keywords(texts, top_n=50):
    all_nouns = []
    for text in texts:
        nouns = extract_nouns(text)
        all_nouns.extend(nouns)
    word_counts = Counter(all_nouns)
    return word_counts.most_common(top_n)

# JSON 파일로 데이터 저장
def save_preview_to_json(df, keywords, output_file="preview.json"):
    preview_data = {
        "keywords": [kw for kw, _ in keywords],
        "articles": {
            keyword: df[df["combined"].str.contains(keyword)].to_dict(orient="records")
            for keyword, _ in keywords
        },
        "company_stats": dict(Counter(df["company"])),
        "keyword_stats": {
            keyword: len(df[df["combined"].str.contains(keyword)]) for keyword, _ in keywords
        },
    }
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(preview_data, f, ensure_ascii=False, indent=4)

    print(f"\nPreview data saved to {output_file}.")

# redis에 데이터 저장
def save_to_redis(df, keywords):
    print("Starting to save data to Redis...")

    # 1. 키워드 목록 저장
    r.delete("keywords")
    r.sadd("keywords", *[kw for kw, _ in keywords])
    print("Keywords saved.")

    # 2. 키워드별 기사 저장
    for keyword, _ in keywords:
        print(f"Saving articles for keyword: {keyword}")
        articles = df[df["combined"].str.contains(keyword)].copy()
        articles = articles.drop(columns=["combined"])

        # 기존 데이터를 삭제하고 새로 추가
        r.delete(f"keyword:{keyword}:articles")
        for _, row in articles.iterrows():
            r.rpush(f"keyword:{keyword}:articles", json.dumps(row.to_dict()))
        print(f"Saved {len(articles)} articles for keyword: {keyword}.")

    # 3. 언론사별 기사 개수 저장 
    print("Saving company stats...")
    r.delete("company:stats")
    company_counts = Counter(df["company"])
    for company, count in company_counts.items():
        r.hset("company:stats", company, count)
    print("Company stats saved.")

    # 4. 키워드별 기사 개수 저장 
    print("Saving keyword stats...")
    r.delete("keyword:stats")
    for keyword, _ in keywords:
        count = len(df[df["combined"].str.contains(keyword)])
        r.hset("keyword:stats", keyword, count)
        print(f"Keyword '{keyword}' has {count} articles.")
    print("Keyword stats saved.")

    print("Data saved to Redis.")

# 실행
if __name__ == "__main__":
    from dotenv import load_dotenv
    load_dotenv()

    # 데이터 로드 및 텍스트 결합
    combined_texts, df = load_and_combine_text(data_folder)

    # 상위 50개 키워드 추출
    top_keywords = extract_top_keywords(combined_texts, top_n=50)
    
    # 상위 키워드 출력
    print("\n상위 50개 키워드:")
    for rank, (word, count) in enumerate(top_keywords, start=1):
        print(f"{rank}. {word} - {count}회 등장")

    # Redis에 저장
    save_to_redis(df, top_keywords)


상위 50개 키워드:
1. 대통령 - 960회 등장
2. 윤석열 - 385회 등장
3. 체포 - 333회 등장
4. 서울 - 316회 등장
5. 영장 - 292회 등장
6. 트럼프 - 232회 등장
7. 미국 - 232회 등장
8. 구속 - 219회 등장
9. 조사 - 204회 등장
10. 공수처 - 204회 등장
11. 내란 - 202회 등장
12. 정부 - 197회 등장
13. 경찰 - 192회 등장
14. 국민 - 189회 등장
15. 집행 - 175회 등장
16. 장관 - 153회 등장
17. 혐의 - 153회 등장
18. 오후 - 146회 등장
19. 시장 - 145회 등장
20. 한국 - 145회 등장
21. 대표 - 137회 등장
22. 심사 - 136회 등장
23. 지원 - 134회 등장
24. 국회 - 134회 등장
25. 비상계엄 - 131회 등장
26. 대행 - 130회 등장
27. 기업 - 127회 등장
28. 서부 - 115회 등장
29. 사업 - 114회 등장
30. 권한 - 113회 등장
31. 수사 - 110회 등장
32. 진행 - 109회 등장
33. 의원 - 109회 등장
34. 지법 - 107회 등장
35. 탄핵 - 107회 등장
36. 경호처 - 106회 등장
37. 사태 - 106회 등장
38. 국가 - 103회 등장
39. 경제 - 102회 등장
40. 범죄 - 101회 등장
41. 사회 - 98회 등장
42. 지역 - 97회 등장
43. 이후 - 97회 등장
44. 경기 - 96회 등장
45. 파이낸셜뉴스 - 95회 등장
46. 계엄 - 95회 등장
47. 출석 - 94회 등장
48. 법원 - 93회 등장
49. 시작 - 91회 등장
50. 기자 - 91회 등장
Starting to save data to Redis...
Keywords saved.
Saving articles for keyword: 대통령
Saved 488 articles for keyword: 대통령.
Saving articles for keywo